## ¿Por que estan dificil saber quien va a ir a Votar?
##Hay dos motivos muy humanos para explicarlo: que las personas somos optimistas en exceso y que nos gusta quedar bien

## https://elpais.com/politica/2019/04/07/actualidad/1554639446_309283.html

## Importamos las librerias necesarias 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor as knnr
import seaborn as sns
sns.set_style("whitegrid")
import os
import pickle

/home/vant/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.



In [3]:
df_2009 = pd.read_csv('../TFM/GDataframe_2009.csv')
df_2009.head()

,Unnamed: 0,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,...,2009_57,2009_58,2006_59,2007,2009_61,2001_62,2010,2009_64,2011_elecc,2011_partic
0,0,48001,Abadiño,43.75,8.65,0.0,185.36,33.4,0.00,4.55,...,10.40,12.90,0.88,12.94,5.86,11.24,7.60,8.50,71.16,2
1,1,20001,Abaltzisketa,25.00,0.00,0.0,269.94,37.6,0.00,0.00,...,92.02,18.69,6.53,10.73,20.61,31.40,22.98,14.93,66.13,2
2,2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,9.60,0.0,250.51,37.0,0.00,2.63,...,14.60,9.29,0.82,8.60,8.15,10.63,21.09,18.13,65.24,2
3,3,20002,Aduna,80.00,36.14,0.0,326.34,23.4,46.62,17.99,...,37.30,12.05,1.59,8.34,29.35,14.69,8.29,24.10,86.46,1
4,4,20016,Aia,33.33,15.63,0.0,114.60,28.5,0.00,5.03,...,9.97,15.63,3.98,20.39,17.74,17.67,11.07,14.97,69.73,2


## Para hacer una prediccion mas realista sobre el porcentaje de participacion electoral en las elecciones municipales en los 245 municipios vascos, vamos a utilizar un numero de participacion entero (tanto como etiqueta para el aprendizaje automatico del modelo de regresion como para el resultado), por ello creamos una funcion.

In [4]:
def part_integ(num):
    try:
        return int(num)
    except:
        return 0

part_integ(67.90)
        

67

In [6]:
df_2009['2011_elecc'] = df_2009['2011_elecc'].apply(part_integ)

In [7]:
df_2009.head()

,Unnamed: 0,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,...,2009_57,2009_58,2006_59,2007,2009_61,2001_62,2010,2009_64,2011_elecc,2011_partic
0,0,48001,Abadiño,43.75,8.65,0.0,185.36,33.4,0.00,4.55,...,10.40,12.90,0.88,12.94,5.86,11.24,7.60,8.50,71,2
1,1,20001,Abaltzisketa,25.00,0.00,0.0,269.94,37.6,0.00,0.00,...,92.02,18.69,6.53,10.73,20.61,31.40,22.98,14.93,66,2
2,2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,9.60,0.0,250.51,37.0,0.00,2.63,...,14.60,9.29,0.82,8.60,8.15,10.63,21.09,18.13,65,2
3,3,20002,Aduna,80.00,36.14,0.0,326.34,23.4,46.62,17.99,...,37.30,12.05,1.59,8.34,29.35,14.69,8.29,24.10,86,1
4,4,20016,Aia,33.33,15.63,0.0,114.60,28.5,0.00,5.03,...,9.97,15.63,3.98,20.39,17.74,17.67,11.07,14.97,69,2


## Por un lado los 64 predictores

In [10]:
X_pred09 = df_2009.drop(['Unnamed: 0','Código municipio', 'Municipio', '2011_elecc','2011_partic'], axis=1).values

## Por otro la etiqueta a utilizar en el modelo

In [11]:
y = df_2009[['2011_elecc']].values

## En el caso de regresiones debemos utilizar arrays por lo que habra que transformar el dataframe de los predictores antes

In [12]:
X = np.array(X_pred09)


 
regressors_names = [
    "Linear Regression", 'K nearest neighbors',
    'Decision Tree Regressor','Random Forest Regressor'
]
 
regressors = [
    LinearRegression(),
    knnr(),
    dtr(),
    RandomForestRegressor()
]

## Vamos a utilizar cuatro modelos de regresion; lineal, K vecinos, Decision Tree y Random Forest. 
## Vamos a entrenar los cuatro modelos con los datos de 2011 (en este caso como tenemos los datos del 2015 para hacer de test nos lo podemos permitir)

In [13]:
lmodel = LinearRegression(n_jobs=1)
lmodel.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
mean_squared_error(df_2009['2011_elecc'],lmodel.predict(X))

27.54713314858559

In [16]:
rmodel=RandomForestRegressor(max_depth=11)
rmodel.fit(X,y)

/home/vant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=11,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
mean_squared_error(df_2009['2011_elecc'],rmodel.predict(X))

7.811775417095629

In [19]:
k_model = knnr(n_neighbors=1)
k_model.fit(X,y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=1, p=2,
          weights='uniform')

In [20]:
mean_squared_error(df_2009['2011_elecc'],k_model.predict(X))

0.0

In [21]:
d_model = dtr(max_depth=11)
d_model.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=11, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [22]:
mean_squared_error(df_2009['2011_elecc'],d_model.predict(X))

1.987424684159378

In [24]:
d_model.predict(X)[0:5]

array([71., 66., 65., 86., 69.])

In [25]:
df_2009['2011_elecc'][0:5]

0    71
1    66
2    65
3    86
4    69
Name: 2011_elecc, dtype: int64

## Una vez entrenado los modelos, testeamos los cuatro modelos con los datos del 2015

## Para comprobar que modelo es el mejor en este caso de regresion una buena metrica a utilizar es la media de error cuadrado, cogiendo el modelo que menor valor que tenga.

In [32]:
df_2013 = pd.read_csv('../TFM/GDataframe_2013.csv')

In [45]:
df_2013['2015_elecc'] = df_2013['2015_elecc'].fillna(71.02)

In [85]:
df_2013['2015_elecc'].describe()

count    245.000000
mean      71.028857
std        8.428252
min       31.370000
25%       64.370000
50%       70.130000
75%       77.930000
max       91.530000
Name: 2015_elecc, dtype: float64

In [46]:
X_pred13 = df_2013.drop(['Unnamed: 0','Código municipio', 'Municipio', '2015_elecc','2015_partic'], axis=1).values

In [47]:
X1 = np.array(X_pred13)

## Es el modelo de regresion de random forest el cual nos da un error cuadratico medio menor, siendo de un 48.33, que teniendo en cuenta que son 245 municipios, con valores de participacion electoral entre 31.37 y 91.53 es un valor razonable.

## Una simple traduccion de esta metrica es que hemos errado 7 puntos de participacion en total sumando los 245 municipios 

In [48]:
mean_squared_error(df_2013['2015_elecc'],rmodel.predict(X1))

48.33020682992144

In [49]:
mean_squared_error(df_2013['2015_elecc'],d_model.predict(X1))

59.642356644916475

In [50]:
mean_squared_error(df_2013['2015_elecc'],k_model.predict(X1))

69.25921918367348

In [51]:
mean_squared_error(df_2013['2015_elecc'],lmodel.predict(X1))

2218679.3554325886

## Ahora utilizamos el modelo entrenado para prediccir la participacion electoral en cada municipio vasco el proximo Mayo del año 2019

In [52]:
df_2017 = pd.read_csv('../TFM/GDataframe_2017.csv')

In [53]:
X_pred17 = df_2017.drop(['Unnamed: 0','Código municipio', 'Municipio'], axis=1).values

In [54]:
X2 = np.array(X_pred17)

In [65]:
pred_19 = pd.DataFrame(rmodel.predict(X2))

In [79]:
pred_19.columns = ['part_pred_2019'] 

In [80]:
Part_pred_19 = pred_19['part_pred_2019'].apply(part_integ)

In [81]:
predicc_17 = pd.read_csv('../TFM/df_foto_pred17.csv')

In [82]:
prediccion_17 = predicc_17.drop('Unnamed: 0', axis=1)

In [83]:
Elecciones_2019 = pd.concat([prediccion_17, Part_pred_19], axis=1)

In [84]:
Elecciones_2019.to_csv('Elecciones_2019.csv')